In [1]:
import pandas as pd
import os
import numpy as np
import time
from multiprocessing.dummy import Pool as ThreadPool
import datetime

In [2]:
calculated_features = r"X:\CIS-PD Study\MJFF Curation\all pre-visit calc features"
watch_accelerometer = r"X:\CIS-PD Study\MJFF Curation\all pre-visit ACC"
combined_data = r"X:\\CIS-PD Study\\MJFF Curation\\combined pre-visit data"

In [3]:
files = os.listdir(calculated_features)
files.insert(0, files.pop(files.index('142620 2017-10-30.csv')))
files.insert(0, files.pop(files.index('142616 2017-10-03.csv')))
files.insert(0, files.pop(files.index('142557 2017-06-29.csv')))
files.insert(0, files.pop(files.index('142616 2018-02-05.csv')))
files.insert(0, files.pop(files.index('142578 2017-09-18.csv')))
print(files)

['142578 2017-09-18.csv', '142616 2018-02-05.csv', '142557 2017-06-29.csv', '142616 2017-10-03.csv', '142620 2017-10-30.csv', '142568 2017-09-18.csv', '142558 2017-07-14.csv', '142609 2017-09-01.csv', '142612 2017-09-06.csv', '142568 2018-02-28.csv', '142615 2017-10-05.csv', '142620 2017-08-10.csv', '142593 2017-11-06.csv', '142601 2017-07-13.csv', '142603 2017-10-13.csv', '142615 2017-07-21.csv', '142558 2017-09-21.csv', '142595 2017-10-12.csv', '142621 2017-10-23.csv', '142608 2017-11-09.csv', '142580 2017-08-01.csv', '142577 2017-11-30.csv', '142603 2017-08-07.csv', '142575 2017-08-29.csv', '142608 2017-09-11.csv', '142619 2017-08-10.csv', '142577 2017-10-11.csv', '142570 2017-07-13.csv', '142604 2017-10-02.csv', '142623 2017-08-03.csv', '142585 2017-09-20.csv', '142583 2017-10-13.csv', '142616 2017-08-01.csv', '142623 2017-10-12.csv', '142604 2017-08-14.csv', '142566 2017-08-09.csv', '142561 2017-07-03.csv', '142619 2017-07-21.csv', '142605 2017-10-11.csv', '142559 2017-07-20.csv',

In [4]:
def drop_Movements_Level(file):
    print(file)
    cf = pd.read_csv(calculated_features + "\\" + file)
    for index, row in cf.iterrows():
        if (row['measurement_id'] == 11):
            cf = cf.drop(index)
    cf.to_csv(calculated_features + "\\" + file)

In [5]:
def drop_extra_index_columns(file):
    print(file)
    cf = pd.read_csv(calculated_features + "\\" + file)
    cols = list(cf.columns.values)
    for col in cols:
        if "Unnamed" in col:
            cf = cf.drop(col, axis = 1)
    cf.to_csv(calculated_features + "\\" + file)

In [6]:
def find_corresponding_cf_data(tm, cf):
    tm = tm - datetime.timedelta(minutes=0,
                             seconds=tm.second % 5,
                             microseconds=tm.microsecond)
    df = cf.loc[cf['timestamp'] == tm]
    return [float(df.loc[(df.measurement_id == 13), 'value']), 
            float(df.loc[(df.measurement_id == 10), 'value']), 
            float(df.loc[(df.measurement_id == 7), 'value'])]
                
                

In [7]:
def merge_file_2(file):
    t1 = time.time()
    print("2:")
    print(file)
    cf = cf = pd.read_csv(calculated_features + "\\" + file)
    wa = pd.read_csv(watch_accelerometer + "\\" + file)
    
    #if os.path.isfile(combined_data + "\\" + file):
    #    return
    
    wa['timestamp'] = pd.to_datetime(wa['timestamp'])
    cf['timestamp'] = pd.to_datetime(cf['timestamp'])
        
    df = wa['timestamp'].apply(lambda tm: pd.Series(find_corresponding_cf_data(tm, cf), 
                                                    index = ['Gait Detection', 
                                                             'Activity Level', 
                                                             'Tremor Score']))
    t_elapsed = ((time.time() - t1)/60.0)
    print(t_elapsed.__str__() + " mins", flush = True)
    return pd.concat([wa, df], axis = 1)

In [8]:
#for file in files:
def merge_file(file):
    t1 = time.time()
    print("1:")
    print(file)
    cf = pd.read_csv(calculated_features + "\\" + file)
    wa = pd.read_csv(watch_accelerometer + "\\" + file)
    
    if os.path.isfile(combined_data + "\\" + file):
        return
    
    wa['timestamp'] = pd.to_datetime(wa['timestamp'])
    #wa['Movements Level'] = np.nan
    wa['Gait Detection'] = np.nan
    wa['Activity Level 5 Seconds'] = np.nan
    wa['Tremor Score'] = np.nan
    
    for index, row in cf.iterrows():
        
        #if (row['measurement_id'] == 11):
        #    continue
        
        start_time = pd.Timestamp(row['timestamp'])
        end_time = start_time
        measurement = row['measurement_name']
        
        value = row['value']
        #if(row['measurement_id'] == 11):
        #    end_time = start_time + pd.Timedelta('00:05:00')
        #else:
        end_time = start_time + pd.Timedelta('00:00:05')
        condition = ((wa.timestamp > start_time) & (wa.timestamp < end_time))
        wa.loc[condition, measurement] = value

    wa.to_csv(combined_data + "\\" + file)

    t_elapsed = ((time.time() - t1)/60.0)
    print(t_elapsed.__str__() + " mins")

In [9]:
files = ['142578 2017-09-18.csv'] #, '142616 2018-02-05.csv', 
         #'142557 2017-06-29.csv', '142616 2017-10-03.csv']

In [ ]:
for file in files:
    merge_file(file)
    df = merge_file_2(file)
    print(df)  
df.style

1:
142578 2017-09-18.csv
2:
142578 2017-09-18.csv


In [14]:
#file = files[0]
#cf = pd.read_csv(calculated_features + "\\" + file)
#wa = pd.read_csv(watch_accelerometer + "\\" + file)

#wa['timestamp'] = pd.to_datetime(wa['timestamp'])
#cf['timestamp'] = pd.to_datetime(cf['timestamp'])

#time = wa.loc[1, 'timestamp']
#tm = datetime.fromtimestamp(time)
#res = find_corresponding_cf_data(time, cf)
#for item in res:
#    print(item)
#print (res)

0.0
0.08473824382398823
0.0870042880649327
[0.0, 0.08473824382398823, 0.0870042880649327]


In [12]:
#pool = ThreadPool(3)
#pool.map(merge_files, files)
#pool.close()
#pool.join()